In [1]:
import scipy.io
import pandas as pd

In [2]:
mat = scipy.io.loadmat('./dataset_3106.mat') 
mat = {k:v for k, v in mat.items() if k[0] != '_'}

In [3]:
mat

{'Y_3106': array([[1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 1, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 1]], dtype=uint8),
 'label_name': array([[array(['Centrosome'], dtype='<U10')],
        [array(['Cytoplasm'], dtype='<U9')],
        [array(['Cytoskeleton'], dtype='<U12')],
        [array(['Endosome'], dtype='<U8')],
        [array(['Endoplasmic-Reticulum'], dtype='<U21')],
        [array(['Extracellular'], dtype='<U13')],
        [array(['Golgi-Apparatus'], dtype='<U15')],
        [array(['Lysosome'], dtype='<U8')],
        [array(['Microsome'], dtype='<U9')],
        [array(['Mitochondrion'], dtype='<U13')],
        [array(['Nucleus'], dtype='<U7')],
        [array(['Peroxisome'], dtype='<U10')],
        [array(['Plasma-Membrane'], dtype='<U15')],
        [array(['Synapse'], dtype='<U7')]], dtype=object),
 'protein_list': array([[array(['Q9Y224'], dtype='<U6'), array(['

In [4]:
data = pd.DataFrame({k: pd.Series(v[0]) for k, v in mat.items()})

In [5]:
data

,Y_3106,label_name,protein_list,sequence_3106
0,1.0,[Centrosome],[Q9Y224],[MFRRKLTALDYHNPAGFNCKDETEFRNFIVWLEDQKIRHYKIEDR...
1,0.0,NaN,[Q5JTW2],NaN
2,0.0,NaN,[Q7Z460],NaN
3,0.0,NaN,[Q5VT06],NaN
4,0.0,NaN,[Q5VYK3],NaN
...,...,...,...,...
3101,NaN,NaN,[P50570],NaN
3102,NaN,NaN,[P78352],NaN
3103,NaN,NaN,[Q9Y4G6],NaN
3104,NaN,NaN,[Q9NRD5],NaN


In [6]:
Y_3106_df = pd.DataFrame(mat['Y_3106'])
label_name_flat = [item[0][0] for item in mat['label_name']]
label_name_df = pd.DataFrame(label_name_flat, columns=['label_name'])
protein_list_flat = [item[0] for item in mat['protein_list'][0]]
protein_list_df = pd.DataFrame(protein_list_flat, columns=['ACC'])
sequence_3106_flat = [item[0][0] for item in mat['sequence_3106']]
sequence_3106_df = pd.DataFrame(sequence_3106_flat, columns=['sequence_3106'])


In [7]:
Y_3106_df.columns = [label_name_df['label_name']]
Y_3106_df['sequence'] = sequence_3106_df['sequence_3106']
Y_3106_df['ACC'] = protein_list_df['ACC']

In [8]:
Y_3106_df.columns

MultiIndex([(           'Centrosome',),
            (            'Cytoplasm',),
            (         'Cytoskeleton',),
            (             'Endosome',),
            ('Endoplasmic-Reticulum',),
            (        'Extracellular',),
            (      'Golgi-Apparatus',),
            (             'Lysosome',),
            (            'Microsome',),
            (        'Mitochondrion',),
            (              'Nucleus',),
            (           'Peroxisome',),
            (      'Plasma-Membrane',),
            (              'Synapse',),
            (             'sequence',),
            (                  'ACC',)],
           names=['label_name'])

In [9]:
Y_3106_df

label_name,Centrosome,Cytoplasm,Cytoskeleton,Endosome,Endoplasmic-Reticulum,Extracellular,Golgi-Apparatus,Lysosome,Microsome,Mitochondrion,Nucleus,Peroxisome,Plasma-Membrane,Synapse,sequence,ACC
0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,MFRRKLTALDYHNPAGFNCKDETEFRNFIVWLEDQKIRHYKIEDRG...,Q9Y224
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,MIDSVKLRRDSAADFFSHYEYLCALQNSVPLPAVRACLREGVLDFN...,Q5JTW2
2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,MEPRMESCLAQVLQKDVGKRLQVGQELIDYFSDKQKSADLEHDQTM...,Q7Z460
3,1,0,0,0,0,0,0,0,0,0,1,0,0,0,MRSSKSKEVPLPNPRNSQSKDTVQADITTSWDALSQTKAALRHIEN...,Q5VT06
4,1,0,0,1,1,0,0,0,0,0,1,0,0,0,MAAAAASASQDELNQLERVFLRLGHAETDEQLQNIISKFLPPVLLK...,Q5VYK3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3101,0,0,0,0,0,0,0,0,0,0,0,0,0,1,MGNRGMEELIPLVNKLQDAFSSIGQSCHLDLPQIAVVGGQSAGKSS...,P50570
3102,0,0,0,0,0,0,0,0,0,0,0,0,0,1,MDCLCIVTTKKYRYQDEDTPPLEHSPAHLPNQANSPPVIVNTDTLE...,P78352
3103,0,0,0,0,0,0,0,0,0,0,0,0,0,1,MVALSLKICVRHCNVVKTMQFEPSTAVYDACRVIRERVPEAQTGQA...,Q9Y4G6
3104,0,0,0,0,0,0,0,0,0,0,0,0,0,1,MFADLDYDIEEDKLGIPTVPGKVTLQKDAQNLIGISIGGGAQYCPC...,Q9NRD5


In [10]:
Y_3106_df.columns.shape[0]-5-2

9

In [11]:
Y_3106_df.to_csv("./Y_3106.csv")

In [12]:
default = pd.read_csv("../multisub_5_partitions_unique.csv")
y_3106_df = pd.read_csv("./Y_3106.csv")

In [13]:
default.columns,y_3106_df.columns

(Index(['Unnamed: 0', 'ACC', 'Kingdom', 'Partition', 'Membrane', 'Cytoplasm',
        'Nucleus', 'Extracellular', 'Cell membrane', 'Mitochondrion', 'Plastid',
        'Endoplasmic reticulum', 'Lysosome/Vacuole', 'Golgi apparatus',
        'Peroxisome', 'Sequence'],
       dtype='object'),
 Index(['label_name', 'Centrosome', 'Cytoplasm', 'Cytoskeleton', 'Endosome',
        'Endoplasmic-Reticulum', 'Extracellular', 'Golgi-Apparatus', 'Lysosome',
        'Microsome', 'Mitochondrion', 'Nucleus', 'Peroxisome',
        'Plasma-Membrane', 'Synapse', 'sequence', 'ACC'],
       dtype='object'))

In [14]:
# Manually removing columns
columns_to_remove_in_y_3106 = [
    'Centrosome',
    'Cytoskeleton',
    'Endosome',
    'Microsome',
    'Synapse',
    'label_name'
]
y_3106_df = y_3106_df.drop(columns_to_remove_in_y_3106,axis=1)


In [15]:
columns_to_rename_in_y_3106 = {
    # old name : new name
    'Endoplasmic-Reticulum' : 'Endoplasmic reticulum',
    'Golgi-Apparatus' : 'Golgi apparatus',
    'Lysosome' : 'Lysosome/Vacuole',
    'Plasma-Membrane' : 'Cell membrane',
}

y_3106_df = y_3106_df.rename(columns=columns_to_rename_in_y_3106)

In [16]:
columns_to_add_in_y_3106 = {
    # column name : default value
    'Kingdom' : '',
    'Partition' : '',
    'Membrane' : 0.0,
    'Plastid' : 0.0,
}
for col in columns_to_add_in_y_3106:
    y_3106_df[col] = columns_to_add_in_y_3106[col]

In [17]:
y_3106_df

,Cytoplasm,Endoplasmic reticulum,Extracellular,Golgi apparatus,Lysosome/Vacuole,Mitochondrion,Nucleus,Peroxisome,Cell membrane,sequence,ACC,Kingdom,Partition,Membrane,Plastid
0,0,0,0,0,0,0,1,0,0,MFRRKLTALDYHNPAGFNCKDETEFRNFIVWLEDQKIRHYKIEDRG...,Q9Y224,,,0.0,0.0
1,0,0,0,0,0,0,0,0,0,MIDSVKLRRDSAADFFSHYEYLCALQNSVPLPAVRACLREGVLDFN...,Q5JTW2,,,0.0,0.0
2,0,0,0,0,0,0,0,0,0,MEPRMESCLAQVLQKDVGKRLQVGQELIDYFSDKQKSADLEHDQTM...,Q7Z460,,,0.0,0.0
3,0,0,0,0,0,0,1,0,0,MRSSKSKEVPLPNPRNSQSKDTVQADITTSWDALSQTKAALRHIEN...,Q5VT06,,,0.0,0.0
4,0,1,0,0,0,0,1,0,0,MAAAAASASQDELNQLERVFLRLGHAETDEQLQNIISKFLPPVLLK...,Q5VYK3,,,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3101,0,0,0,0,0,0,0,0,0,MGNRGMEELIPLVNKLQDAFSSIGQSCHLDLPQIAVVGGQSAGKSS...,P50570,,,0.0,0.0
3102,0,0,0,0,0,0,0,0,0,MDCLCIVTTKKYRYQDEDTPPLEHSPAHLPNQANSPPVIVNTDTLE...,P78352,,,0.0,0.0
3103,0,0,0,0,0,0,0,0,0,MVALSLKICVRHCNVVKTMQFEPSTAVYDACRVIRERVPEAQTGQA...,Q9Y4G6,,,0.0,0.0
3104,0,0,0,0,0,0,0,0,0,MFADLDYDIEEDKLGIPTVPGKVTLQKDAQNLIGISIGGGAQYCPC...,Q9NRD5,,,0.0,0.0


In [18]:
default =default.drop('Unnamed: 0',axis=1)

In [19]:
default.columns.shape[0],y_3106_df.columns.shape[0]

(15, 15)

In [20]:
combined_df = pd.concat([default,y_3106_df],ignore_index=True)

In [21]:
combined_df.to_csv("./default_updated_Y3106.csv")